In [2]:
!nvidia-smi


Sun Jan 18 13:19:37 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
%%writefile task1.cu
#include <cuda_runtime.h>
#include <iostream>
#include <chrono>

#define N 1000000  // размер массива
#define BLOCK_SIZE 256

//
// ЯДРО 1: Используем только ГЛОБАЛЬНУЮ память
//
__global__ void multiply_global(float* data, float value) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // проверяем, чтобы не выйти за границы массива
    if (idx < N) {
        data[idx] *= value;
    }
}

//
// ЯДРО 2: Используем SHARED MEMORY
//
__global__ void multiply_shared(float* data, float value) {
    // разделяемая память внутри блока
    __shared__ float shared_data[BLOCK_SIZE];

    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int tid = threadIdx.x;

    // копируем данные из глобальной памяти в shared
    if (idx < N) {
        shared_data[tid] = data[idx];
    }

    // ждём, пока все потоки блока загрузят данные
    __syncthreads();

    // выполняем умножение
    if (idx < N) {
        shared_data[tid] *= value;
    }

    // снова ждём все потоки
    __syncthreads();

    // записываем результат обратно в глобальную память
    if (idx < N) {
        data[idx] = shared_data[tid];
    }
}

int main() {
    float* h_data = new float[N];
    float* d_data;

    // инициализация массива на CPU
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    // выделяем память на GPU
    cudaMalloc(&d_data, N * sizeof(float));

    // копируем данные с CPU на GPU
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    dim3 block(BLOCK_SIZE);
    dim3 grid((N + BLOCK_SIZE - 1) / BLOCK_SIZE);

    //
    // Замер ГЛОБАЛЬНОЙ памяти
    //
    auto start_global = std::chrono::high_resolution_clock::now();

    multiply_global<<<grid, block>>>(d_data, 2.0f);
    cudaDeviceSynchronize();

    auto end_global = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> time_global = end_global - start_global;

    //
    // Замер SHARED MEMORY
    //
    auto start_shared = std::chrono::high_resolution_clock::now();

    multiply_shared<<<grid, block>>>(d_data, 2.0f);
    cudaDeviceSynchronize();

    auto end_shared = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> time_shared = end_shared - start_shared;

    // копируем результат обратно на CPU
    cudaMemcpy(h_data, d_data, N * sizeof(float), cudaMemcpyDeviceToHost);

    // вывод результатов
    std::cout << "Время (глобальная память): " << time_global.count() << " секунд\n";
    std::cout << "Время (shared память):     " << time_shared.count() << " секунд\n";

    // очистка памяти
    cudaFree(d_data);
    delete[] h_data;

    return 0;
}


Writing task1.cu


In [6]:
!nvcc task1.cu -o task1
# Компиляция

In [7]:
!./task1
# Запуск


Время (глобальная память): 0.00745882 секунд
Время (shared память):     2.484e-06 секунд
